In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)
from plotly.offline import iplot, init_notebook_mode
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go
import re


## 數據前處理階段

In [ ]:
# sales = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='出貨明細') # Mac使用
sales = pd.read_excel(r'C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx', sheet_name='出貨明細')
# budget = pd.read_excel('/Users/kai/Desktop/Weekly report_v0.1.xlsx', sheet_name='2020預算', usecols='A:F') # Mac使用
# budget = pd.read_excel(r'C:\Users\kaihsu\Desktop\業績總表\Weekly report_v0.1.xlsx', sheet_name='2020預算', usecols='A:F')

In [ ]:
# sales['Size'] = sales['產品分類'].str.extract('([0-9]+)\w', expand=True)

In [ ]:
sales = sales.query('預交年份 == (2019,2020)')

In [ ]:
sales['預交日期'] = sales['預交日期'].apply(lambda x:x.strftime('%Y-%m'))

In [ ]:
df = sales.groupby(['Group','預交日期'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
df

In [ ]:
df[['數量', '本國幣別NTD']].describe()

In [ ]:
df = df.query('Group == "華新科技"')
fig_total = px.scatter(df, x='數量', y='本國幣別NTD',
                       size='本國幣別NTD', text='Group',
                       color='Group',
                       animation_frame='預交日期',
                       range_x=[-10, 30000000],
                       range_y=[0, 3000000],
                       color_discrete_sequence=px.colors.qualitative.D3,
                       size_max=50,
                       hover_name='Group'
                       )
fig_total.show()

In [ ]:
sales['品名'].str.extract('(^\D+[0-9]+)([A-Z]+)')

In [ ]:
budget.insert(2, 'BU', value=budget.apply(lambda x: 'RF' if 'RF' in x['客戶名稱'] else 'Component', axis=1))

In [ ]:
budget['客戶名稱'] = budget['客戶名稱'].str.replace(' RF', '')

In [ ]:
client_dict = {'Bourns': 'Bourns', 'Eaton': 'Eaton', 'Laird': 'Laird', 'LF': 'LF', 'LF-OEM': 'LF ORM', 'WE': 'WE',
               '信昌': '信昌', '東莞弘電': '東莞弘電', '藝感': '藝感', '華科': '華新科技',
               'Mt System(Component)': 'MT-SYSTEM', 'Teltonika(Component)': 'TELTONIKA', 'Mt System(RF)': 'MT-SYSTEM',
               'Teltonika(RF)': 'TELTONIKA', 'Te(RF)': 'TE', 'Credence(India)': 'Credence',
               'Millenium(India)': 'Millenium', 'Nexty(India)': 'Nexty', 'OSE(India)': 'OSE', 'Others/One Time PO(India)': 'Others/One Time PO(India)'}
budget['客戶名稱'] = budget['客戶名稱'].map(client_dict)

In [ ]:
sales = sales[~sales['狀態'].str.contains('待通知 | 合約')]

## 隨便亂打區

In [ ]:
x = sales.groupby(['預交年份', '預交月份', '負責業務', 'BG'])[['本國幣別NTD']].sum().reset_index()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
x['預交月份'] = pd.Categorical(x['預交月份'], ordered=True, categories=month_order)
x = x.sort_values(by='預交月份').reset_index()

In [ ]:
df = sales[['BG', 'Group', '狀態', '開單日期', '預交日期', '負責業務', '品名', '數量', '本國幣別NTD', '客戶料號']]

In [ ]:
test = pd.pivot_table(df, index=['負責業務', 'Group', 'BG'], values=['本國幣別NTD', '數量'], aggfunc={'本國幣別NTD': sum, '數量': sum}).reset_index()

In [ ]:
test = test.style.format({'本國幣別NTD':'{0:,.0f}'}).highlight_max(color='green')

In [ ]:
x = test['負責業務'].tolist()
y = test['本國幣別NTD'].tolist()

In [ ]:
from plotly.subplots import make_subplots

# fig = make_subplots(rows=1, cols=2)

# fig = ff.create_table(test, height_constant=60)

fig = px.treemap(test, path=['負責業務', 'Group'], values='本國幣別NTD')
# dict_fig = dict({'data': [{'type': 'bar', 'x': test['負責業務'], 'y': test['本國幣別NTD']}]})
# fig.add_trace(go.Bar(x=test['負責業務'], y=test['本國幣別NTD'], xaxis='x2', yaxis='y2'))
fig.add_trace(go.Bar(x=test['負責業務'], y=test['本國幣別NTD'], xaxis='x2', yaxis='y2'))

# fig.add_trace(go.Treemap(labels=test['負責業務'],
#                 parents=test['Group'],
#                 values=test['本國幣別NTD'], row=1, col=2))

              
# '''
# 追加的圖表用update_layout方法增加，
# 裡面需要用xaxis的位置和新增圖表，xaxis2的位置
# '''
              
fig.update_layout(title_text='測試圖表',
                  height= 1080, width= 1920,
                 xaxis = {'domain': [0, 0.6]},
                 xaxis2 = {'domain': [0.7, 1]},
                 yaxis2 = {'anchor': 'x2', 'title': 'Test'})

fig.show()


In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/sales_success.csv')

In [ ]:
levels = ['salesperson', 'county', 'region'] # levels used for the hierarchical chart
color_columns = ['sales', 'calls']
value_column = 'calls'

def build_hierarchical_dataframe(df, levels, value_column, color_columns=None):
    """
    Build a hierarchy of levels for Sunburst or Treemap charts.

    Levels are given starting from the bottom to the top of the hierarchy,
    ie the last level corresponds to the root.
    """
    df_all_trees = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
    for i, level in enumerate(levels):
        df_tree = pd.DataFrame(columns=['id', 'parent', 'value', 'color'])
        dfg = df.groupby(levels[i:]).sum()
        dfg = dfg.reset_index()
        df_tree['id'] = dfg[level].copy()
        if i < len(levels) - 1:
            df_tree['parent'] = dfg[levels[i+1]].copy()
        else:
            df_tree['parent'] = 'total'
        df_tree['value'] = dfg[value_column]
        df_tree['color'] = dfg[color_columns[0]] / dfg[color_columns[1]]
        df_all_trees = df_all_trees.append(df_tree, ignore_index=True)
    total = pd.Series(dict(id='total', parent='',
                              value=df[value_column].sum(),
                              color=df[color_columns[0]].sum() / df[color_columns[1]].sum()))
    df_all_trees = df_all_trees.append(total, ignore_index=True)
    return df_all_trees


df_all_trees = build_hierarchical_dataframe(df, levels, value_column, color_columns)
average_score = df['sales'].sum() / df['calls'].sum()

In [ ]:
fig = go.Treemap(labels=test['負責業務'],
                parents=test['Group'],
                values=test['本國幣別NTD'])
fig

## Treemap

In [ ]:
sales[(sales['負責業務'] == '許凱智') & (sales['預交年份'].isin([2019,2020])) & (sales['Group'].isin(['TELTONIKA', 'MT-SYSTEM']))]

In [ ]:
test = sales[(sales['負責業務'] == '許凱智') & (sales['預交年份'].isin([2019,2020])) & (sales['Group'].isin(['TELTONIKA', 'MT-SYSTEM']))].groupby(['預交年份', 'Group', 'BG', '預交月份', '品名'])[['數量', '本國幣別NTD']].sum().reset_index()

In [ ]:
test.columns = ['Year', 'Company', 'Category', 'Month', 'Part number', 'Quantity', 'Revenue in NTD']

In [ ]:
data = test.query('Year == 2020 and Company == "MT-SYSTEM" and Category == "RF"')
data = data[['Quantity']]

In [ ]:
test1 = test.groupby(['Year', 'Company', 'Month', 'Category', 'Part number'])[['Quantity', 'Revenue in NTD']].sum().reset_index()

In [ ]:
fig1 = px.treemap(test1, path=['Year', 'Company', 'Month', 'Category', 'Part number'], values='Revenue in NTD',
                color='Quantity', color_continuous_scale='RdBu'
                )
fig1.show()
with open('MT-SYSTEM.html', 'w') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()

In [ ]:
fig = px.treemap(test, path=['預交年份', '預交月份', '負責業務', 'Group', 'BG'], values='本國幣別NTD',
                color='數量', color_continuous_scale='RdBu'
                )
fig.show()

In [ ]:
test1 = test.query('Group == ["TELTONIKA", "MT-SYSTEM"]')
fig = px.treemap(test1, path=['預交年份', 'Group', 'BG'], values='本國幣別NTD',
                color='數量', color_continuous_scale='RdBu'
                )
fig.show()

## Line plot

In [ ]:
from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# ss.fit_transform(x['本國幣別NTD'])
x['std'] = preprocessing.scale(x['本國幣別NTD'])

In [ ]:
hist = px.histogram(x, x='預交月份', y='本國幣別NTD', facet_col='預交年份', facet_row='負責業務',
                    category_orders={'預交年份':[2019,2020,2021,2022]},
                   labels={'本國幣別NTD':'營業額', '預交月份':'月份'})
hist.show()

In [ ]:
line = px.line(x, x='預交月份', y='本國幣別NTD',color='預交年份', category_orders={'預交月份':month_order}, facet_col='負責業務', facet_row='BG')
line.show()

In [ ]:
line_plot = test.groupby(['預交年份', '預交月份', '負責業務'])[['本國幣別NTD']].sum().reset_index()
month_order = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
               'August', 'September', 'October', 'November', 'December']
line_plot['預交月份'] = pd.Categorical(line_plot['預交月份'], ordered=True, categories=month_order)
line_plot = line_plot.sort_values(by='預交月份')

In [ ]:
line = px.line(line_plot, x='預交月份', y='本國幣別NTD', color='預交年份', category_orders={'預交月份':month_order}, facet_col='負責業務')
line.show()

## Bar plot

In [ ]:
df = test.query('預交年份 == (2020, 2019, 2021)')
df = df.groupby(['預交年份', '負責業務', '預交月份'])[['本國幣別NTD']].sum().reset_index()
df['預交年份'] = df['預交年份'].astype(str)
bar = px.bar(df, x='預交月份', y='本國幣別NTD', color='預交年份', barmode='group',
             category_orders={'預交月份':month_order}, facet_col='負責業務'
            )
bar.show()

## Box Plot


In [ ]:
data = sales.query('預交年份 == 2020')
data = data.groupby(['銷售單號', 'Group'])[['本國幣別NTD']].sum().reset_index()
fig = px.box(data, x='Group', y= '本國幣別NTD')
fig.show()

## Histogram

In [ ]:
df = sales.query('預交年份 == 2020 and Group == "Eaton"')
df = df.groupby(['銷售單號'])[['本國幣別NTD']].sum().reset_index()

In [ ]:

fig = px.histogram(df, x='本國幣別NTD')
fig.show()

In [ ]:
from sklearn import preprocessing

df['std'] = preprocessing.scale(df['本國幣別NTD'])

In [ ]:
df['本國幣別NTD'].describe()

In [ ]:
import plotly.figure_factory as ff

hist_data = [df['std']]
# fig = ff.create_distplot(hist_data, group_labels='Test')
hist_data


## 將多個figures結合成一份HTML

In [ ]:
with open('test.html', 'a') as f:
    f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(line.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(bar.to_html(full_html=False, include_plotlyjs='cdn'))
    f.close()